In [1]:
# import utilities libraries
import os 
import cv2 
import numpy as np 
from pathlib import Path
from tqdm import tqdm
import shutil
import glob
import random

# import yolo model
from ultralytics import YOLO
import torch

In [2]:
model = YOLO("yolo11s.pt")
model.info()

YOLO11s summary: 181 layers, 9,458,752 parameters, 0 gradients, 21.7 GFLOPs


(181, 9458752, 0, 21.718374400000002)

In [3]:
# --- CONFIG ---
DEST_ROOT = Path("yolo_dataset_final")
CLASS_MAP = {'gl': 0, 'me': 1, 'pi': 2}

def find_folder(name):
    cur = Path.cwd()
    if (cur / name).exists(): return cur / name
    if (cur / "brisc2025" / name).exists(): return cur / "brisc2025" / name
    found = list(cur.rglob(name))
    return found[0] if found else None

def get_yolo_box(mask_path, class_id):
    mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
    if mask is None: return None
    _, thresh = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    labels = []
    h, w = mask.shape[:2]
    for cnt in contours:
        x, y, w_box, h_box = cv2.boundingRect(cnt)
        if w_box < 3 or h_box < 3: continue 
        labels.append(f"{class_id} {(x + w_box/2)/w:.6f} {(y + h_box/2)/h:.6f} {w_box/w:.6f} {h_box/h:.6f}")
    return labels

def get_id(fname):
    n = fname.lower()
    if 'glioma' in n or '_gl_' in n: return 0
    if 'meningioma' in n or '_me_' in n: return 1
    if 'pituitary' in n or '_pi_' in n: return 2
    return None

def build_dataset():
    if DEST_ROOT.exists(): shutil.rmtree(DEST_ROOT)
    
    seg_root = find_folder("segmentation_task")
    cls_root = find_folder("classification_task")
    
    for split in ["train", "test"]:
        (DEST_ROOT/ "images" / split).mkdir(parents=True, exist_ok=True)
        (DEST_ROOT/ "labels" / split).mkdir(parents=True, exist_ok=True)

        # 1. PROCESS TUMORS (From Segmentation)
        print(f"Processing {split} Tumors...")
        img_src = seg_root / split / ("images" if (seg_root/split/"images").exists() else "image")
        mask_src = seg_root / split / ("masks" if (seg_root/split/"masks").exists() else "mask")
        
        for img_path in tqdm(list(img_src.glob("*.*"))):
            if img_path.suffix not in ['.jpg', '.png', '.jpeg']: continue
            cid = get_id(img_path.name)
            if cid is None: continue

            mask_p = mask_src / img_path.name
            if not mask_p.exists(): mask_p = mask_src / img_path.with_suffix('.png').name
            
            if mask_p.exists():
                lbls = get_yolo_box(mask_p, cid)
                if lbls:
                    shutil.copy(img_path, DEST_ROOT/"images"/split/img_path.name)
                    with open(DEST_ROOT/"labels"/split/img_path.with_suffix('.txt').name, "w") as f:
                        f.write("\n".join(lbls))

        # 2. PROCESS NO-TUMOR (From Classification)
        print(f"Adding {split} Healthy Backgrounds...")
        nt_src = cls_root / split / "no_tumor"
        if nt_src.exists():
            for img_path in tqdm(list(nt_src.glob("*.*"))):
                if img_path.suffix not in ['.jpg', '.png', '.jpeg']: continue
                new_name = f"bg_{img_path.name}"
                shutil.copy(img_path, DEST_ROOT/"images"/split/new_name)
                # Empty label file = No Tumor
                with open(DEST_ROOT/"labels"/split/(Path(new_name).stem+".txt"), "w") as f: pass

    # Config
    with open("data_final.yaml", "w") as f:
        f.write(f"path: {DEST_ROOT.resolve().as_posix()}\ntrain: images/train\nval: images/test\nnc: 3\nnames: ['glioma', 'meningioma', 'pituitary']")

if __name__ == "__main__":
    build_dataset()

Processing train Tumors...


100%|██████████| 3933/3933 [00:15<00:00, 252.80it/s]


Adding train Healthy Backgrounds...


100%|██████████| 1067/1067 [00:01<00:00, 555.13it/s]


Processing test Tumors...


100%|██████████| 860/860 [00:03<00:00, 258.12it/s]


Adding test Healthy Backgrounds...


100%|██████████| 140/140 [00:00<00:00, 571.14it/s]


In [4]:
def train_ultimate():
    # Load the SMALL Architecture
    model = YOLO("yolo11s-bifpn.yaml") 

    # Transfer Learning
    try: model.load("yolo11s.pt") 
    except: pass

    # Train
    model.train(
        data="data_final.yaml",
        epochs=100,               
        imgsz=512,                
        batch=2,                 
        
        device=0,
        workers=0,
        project="BrainTumor_Final",
        name="run_93percent_target",
        
        # Optimizer Tweaks for Accuracy
        optimizer="AdamW",
        lr0=0.001,
        lrf=0.05,                  
        cos_lr=True,
        
        # Augmentation 
        degrees=10.0,              # Rotation
        flipud=0.5,                # Up-down flip (brains are symmetric-ish)
        fliplr=0.5,
        mosaic=1.0,
        mixup=0.1,
        
        amp=True,
        exist_ok=True
    )

if __name__ == "__main__":
    torch.multiprocessing.freeze_support()
    train_ultimate()

Transferred 196/586 items from pretrained weights
New https://pypi.org/project/ultralytics/8.3.237 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.233  Python-3.11.14 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=2, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data_final.yaml, degrees=10.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.05, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolo11s-bifpn.yaml, momentum=0.937, mosaic=1.0, m

KeyboardInterrupt: 

In [5]:
# Path to your final weights
MODEL_PATH = r"BrainTumor_Final\run_93percent_target\weights\best.pt"

NO_TUMOR_FOLDER = r"brisc2025\classification_task\test\no_tumor" 
TUMOR_FOLDER = r"yolo_dataset_multiclass\images\test"

def test_model():
    model = YOLO(MODEL_PATH)
    
    # 1. Test a HEALTHY Brain (Should detect NOTHING)
    print("\n--- TEST 1: Healthy Brain (Expect No Detections) ---")
    nt_files = list(Path(NO_TUMOR_FOLDER).glob("*.jpg"))
    if nt_files:
        img = random.choice(nt_files)
        results = model.predict(img, conf=0.5, save=False) # High confidence threshold
        
        if not results[0].boxes:
            print(f"Model looked at {img.name} and found nothing.")
        else:
            print(f"False Positive on {img.name}.")
            
        # Show it
        im_array = results[0].plot()
        cv2.imshow("Healthy Brain Test", im_array)
        cv2.waitKey(0)
    
    # 2. Test a TUMOR Brain (Should detect Class 0, 1, or 2)
    print("\n--- TEST 2: Tumor Brain (Expect Detection) ---")
    t_files = list(Path(TUMOR_FOLDER).glob("*.jpg"))
    if t_files:
        img = random.choice(t_files)
        results = model.predict(img, conf=0.25, save=False)
        
        if results[0].boxes:
            cls_id = int(results[0].boxes.cls[0].item())
            name = results[0].names[cls_id]
            print(f"Detected {name} in {img.name}")
        else:
            print(f"Failed to find tumor in {img.name}")

        im_array = results[0].plot()
        cv2.imshow("Tumor Test", im_array)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

if __name__ == "__main__":
    test_model()


--- TEST 1: Healthy Brain (Expect No Detections) ---

image 1/1 d:\Data Adji\Binus\Semester 5\DeepLearning\Project\brisc2025\classification_task\test\no_tumor\brisc2025_test_00700_no_sa_t1.jpg: 512x512 (no detections), 132.2ms
Speed: 5.2ms preprocess, 132.2ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 512)
Model looked at brisc2025_test_00700_no_sa_t1.jpg and found nothing.

--- TEST 2: Tumor Brain (Expect Detection) ---

image 1/1 d:\Data Adji\Binus\Semester 5\DeepLearning\Project\yolo_dataset_multiclass\images\test\brisc2025_test_00884_pi_co_t1.jpg: 512x512 1 pituitary, 139.9ms
Speed: 4.5ms preprocess, 139.9ms inference, 3.6ms postprocess per image at shape (1, 3, 512, 512)
Detected pituitary in brisc2025_test_00884_pi_co_t1.jpg


In [6]:
def export_onnx():
    # 1. Path to your best weights from the 100-epoch run
    model_path = r"BrainTumor_Final\run_93percent_target\weights\best.pt"
    
    if not os.path.exists(model_path):
        print(f"Error: Could not find model at {model_path}")
        return

    print(f"Loading model from: {model_path}")
    model = YOLO(model_path)

    # 2. Export
    # format='onnx': The target format
    # opset=12: Most compatible version for ONNX Runtime (Vercel/Web)
    # dynamic=False: Fixes input size to 512x512 (faster/simpler for web apps)
    success = model.export(format="onnx", opset=12, dynamic=False)
    
    print(f"\nComplete: {success}")
    print("best.onnx created")

if __name__ == "__main__":
    export_onnx()

Loading model from: BrainTumor_Final\run_93percent_target\weights\best.pt
Ultralytics 8.3.233  Python-3.11.14 torch-2.5.1 CPU (AMD Ryzen 7 4800H with Radeon Graphics)
YOLO11s-bifpn summary (fused): 122 layers, 5,198,907 parameters, 0 gradients, 25.0 GFLOPs

PyTorch: starting from 'BrainTumor_Final\run_93percent_target\weights\best.pt' with input shape (1, 3, 512, 512) BCHW and output shape(s) (1, 7, 21504) (10.3 MB)

ONNX: starting export with onnx 1.19.1 opset 12...
ONNX: slimming with onnxslim 0.1.77...
ONNX: export success  3.2s, saved as 'BrainTumor_Final\run_93percent_target\weights\best.onnx' (16.5 MB)

Export complete (3.9s)
Results saved to D:\Data Adji\Binus\Semester 5\DeepLearning\Project\BrainTumor_Final\run_93percent_target\weights
Predict:         yolo predict task=detect model=BrainTumor_Final\run_93percent_target\weights\best.onnx imgsz=512  
Validate:        yolo val task=detect model=BrainTumor_Final\run_93percent_target\weights\best.onnx imgsz=512 data=data_final.yaml